# Importing Important libraries and directory name etc.

In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
%matplotlib inline
import seaborn as sns # for advanced data visualizations
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the data set:

In [48]:
vgSales = pd.read_csv("../input/videogamesales/vgsales.csv")
vgSales.head()

Now, we have imported the data set. We ready to go for the data cleaning and the lifescale of a data science project.

### To get basic onformation of the data set:

In [49]:
vgSales.info()

To get the basic data of all the columns like the non-null, count as well as data type. As we can see there are 271 records in "Year" column and 58 records in "Publisher". There are total of 16598 records in this dataset.

Also, it is displaying the data types which is very important to know before applying any functions or even before working with the data.

We can see that "Year" column is in float64 which means that years are stores in this format ---> 2018.0 or 2021.0. We have to convert them into int64 format.

In [50]:
vgSales.describe()

With this function describe(), we get to know the count, mean, standard deviation, minimum, maximum etc.

## To check our data is clean or not?

**Cleaning includes normalization as well as removal of NA Values in the data set. Basically removing the Noise in the data set.**

In [51]:
vgSales.isna().sum()

In [52]:
vgSales.isna().sum().sum()

There are aroud 271 records in ***Year*** column and around 58 records in ***Publisher*** column whoch are NA Value records.
***
We need to remove these before the visualization and taking useful insights from them.

In [53]:
def dfCategory(dFrame):
    for i in dFrame.columns:
        print(dFrame.groupby(i)[i].count())
        print('\n')

In [54]:
dfCategory(vgSales)

It basically gives the count of each categorical variable or records in the data set.

We are going to check the numerical data in the data set for normalization.

In [55]:
print("The Skew Count of NA_Sales Column is:",vgSales["NA_Sales"].skew())
print("The Skew Count of EU_Sales Column is:",vgSales["EU_Sales"].skew())
print("The Skew Count of JP_Sales Column is:",vgSales["JP_Sales"].skew())
print("The Skew Count of Other_Sales Column is:",vgSales["Other_Sales"].skew())

So, we are normalizing the data with the help of Square Root Normalization Method.

In [56]:
vgSales["NA_Sales"] = vgSales["NA_Sales"]**(1/2)
vgSales["EU_Sales"] = vgSales["EU_Sales"]**(1/2)
vgSales["JP_Sales"] = vgSales["JP_Sales"]**(1/2)
vgSales["Other_Sales"] = vgSales["Other_Sales"]**(1/2)
vgSales["Global_Sales"] = vgSales["NA_Sales"] + vgSales["EU_Sales"] + vgSales["JP_Sales"] + vgSales["Other_Sales"]

In [57]:
vgSales

***Now let's check the Anamolies of the data:***

In [58]:
print("Max Year Value: ", vgSales['Year'].max())

The data set should only contain values or records till 2017 year not till 2020.

In [59]:
maxEntry = vgSales['Year'].idxmax()
vgSales.iloc[maxEntry]

This game was released on 2009 year and not in 2020.
[https://www.google.com/search?q=Imagine%3A+Makeup+Artist&rlz=1C1VIQF_enIN976IN976&oq=imagine+&aqs=chrome.2.69i57j0i433i512j69i59j46i433i512j0i433i457i512j0i402j0i433i512l2j0i512j0i433i512.3656j0j7&sourceid=chrome&ie=UTF-8](http://)

You can check the link above for the info on this game.

In [60]:
vgSales["Year"] = vgSales["Year"].replace(2020.0, 2009.0)

In [61]:
print("Max Year Value: ", vgSales["Year"].max())

Now, the max year is 2017 which is better and apt.
***
We are checking anymore anamoly such as the above exists in our data set.

In [62]:
YearAnamoly = vgSales[vgSales['Year'].isnull()]['Name'].unique()
print("The year records having such anamoly: ", len(YearAnamoly))

***For such Anamolies, we thought of replaxing them with 2009 year most of the games were produced in the year 2009 according to this data set:***

In [63]:
plt.figure(figsize = (20,20))
plt.bar(vgSales["Year"].value_counts().index, vgSales["Year"].value_counts())
plt.title("The most games produced in a specific year")
plt.show()

The above graph shows that most of the games were released in 2009 and 2010.

In [64]:
vgSales['Year'] = vgSales['Year'].fillna(2009.0)
vgSales['Year'].isnull().sum()

We have successfully removed the NA Records and Anamolies in the Year column.
***
Now, converting year from Float64 -> Int64

In [65]:
vgSales['Year'] = vgSales['Year'].astype('int64')
vgSales.head()

We removed the 58 NA Records in Publisher because it won't affect the visualization as it was an unnescessary noise in the data.

In [66]:
gameNames = vgSales['Name'].unique()
publisherNames = vgSales['Publisher'].unique()
platformNames = vgSales['Platform'].unique()
genreNames = vgSales['Genre'].unique()
print("There are",len(gameNames),"games in the data set.")
print("There are",len(publisherNames),"publishers in the data set.")
print("There are",len(platformNames),"platforms in the data set.")
print("There are",len(genreNames),"different genres in the data set.")

## Exploratory Data Analysis:

### To get to know which genre sold the most globally:

In [67]:
vgSales[["Genre","Global_Sales"]].groupby("Genre").sum()

The most sold genre of games globally is ***Action Genre***. The second highest selling genre globally is ***Sports Genre***.

### To get the most games produced in a specific Gaming Platform:

In [68]:
plt.figure(figsize = (20,20))
plt.bar(vgSales["Platform"].value_counts().index, vgSales["Platform"].value_counts())
plt.title("Most Games produced in Specific Gaming Platform")
plt.show()

We can see that DS and PS2 are tied for the topspot in this